In [101]:
import rasterio
import rioxarray
import dask
import xarray as xr
xr.set_options(use_flox=True)

from tqdm import tqdm

In [102]:
geotiff_path = "GHS_POP_E2025_GLOBE_R2023A_4326_30ss_V1_0.tif"
population_dataarray = rioxarray.open_rasterio(geotiff_path, masked=True, chunks=True)
population_dataarray

<xarray.DataArray (band: 1, y: 21384, x: 43202)> Size: 7GB
dask.array<open_rasterio-b5388ab6ddcedb23ff91718a0557dd18<this-array>, shape=(1, 21384, 43202), dtype=float64, chunksize=(1, 4096, 4096), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 171kB 89.1 89.09 89.08 ... -89.08 -89.09 -89.1
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        286330.44519772
    STATISTICS_MEAN:           8.8674048604304
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         249.62729204313
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

### Map to weather grid points
Now we want to do perform a convolution to get it to a desired resolution. In our case, we want to convolve it such that the result aligns with a given weather grid.
We would start with NOAA data (i.e. 0.25 degrees but we need to make sure they align).

-> Better: group the points by the closest point in the weather dataset and sum over them. 

In [117]:
weather_dataset = xr.open_dataset('HRRR_reprojected.grib2', engine='rasterio')
weather_dataset

<xarray.Dataset> Size: 595MB
Dimensions:      (band: 29, x: 2441, y: 1050)
Coordinates:
  * band         (band) int64 232B 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29
  * x            (x) float64 20kB -134.1 -134.1 -134.0 ... -60.97 -60.94 -60.91
  * y            (y) float64 8kB 52.6 52.57 52.54 52.51 ... 21.19 21.16 21.13
    spatial_ref  int64 8B ...
Data variables:
    band_data    (band, y, x) float64 595MB ...

In [118]:
# weather_dataset = xr.open_dataset('NOAA_2024_01_01_0_000.grib2', engine='rasterio')
# weather_dataset

In [119]:
x_indexer = weather_dataset.x.to_index()[weather_dataset.x.to_index().get_indexer(population_dataarray.x, method='nearest')]
y_indexer = weather_dataset.y.to_index()[weather_dataset.y.to_index().get_indexer(population_dataarray.y, method='nearest')]

# Add these indices as new coordinates to data_array.
# This doesn't move any data, just adds metadata.
population_dataarray.coords['weather_x_idx'] = ('x', x_indexer)
population_dataarray.coords['weather_y_idx'] = ('y', y_indexer)

print("Added 'weather_x_idx' and 'weather_y_idx' to data_array.coords")
# You can inspect data_array now to see the new coordinates.

if 'band' in population_dataarray.dims and population_dataarray.sizes['band'] == 1:
    population_dataarray = population_dataarray.squeeze('band', drop=True)
population_dataarray

Added 'weather_x_idx' and 'weather_y_idx' to data_array.coords


<xarray.DataArray (y: 21384, x: 43186)> Size: 7GB
dask.array<getitem, shape=(21384, 43186), dtype=float64, chunksize=(4096, 4096), chunktype=numpy.ndarray>
Coordinates:
  * x              (x) float64 345kB -180.0 -180.0 -180.0 ... 179.9 179.9 179.9
  * y              (y) float64 171kB 89.1 89.09 89.08 ... -89.08 -89.09 -89.1
    spatial_ref    int64 8B 0
    weather_x_idx  (x) float64 345kB -134.1 -134.1 -134.1 ... -60.91 -60.91
    weather_y_idx  (y) float64 171kB 52.6 52.6 52.6 52.6 ... 21.13 21.13 21.13
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        286330.44519772
    STATISTICS_MEAN:           8.8674048604304
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         249.62729204313
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

### Align population grid with resolution of weather grid

In [120]:
population_dataarray

<xarray.DataArray (y: 21384, x: 43186)> Size: 7GB
dask.array<getitem, shape=(21384, 43186), dtype=float64, chunksize=(4096, 4096), chunktype=numpy.ndarray>
Coordinates:
  * x              (x) float64 345kB -180.0 -180.0 -180.0 ... 179.9 179.9 179.9
  * y              (y) float64 171kB 89.1 89.09 89.08 ... -89.08 -89.09 -89.1
    spatial_ref    int64 8B 0
    weather_x_idx  (x) float64 345kB -134.1 -134.1 -134.1 ... -60.91 -60.91
    weather_y_idx  (y) float64 171kB 52.6 52.6 52.6 52.6 ... 21.13 21.13 21.13
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        286330.44519772
    STATISTICS_MEAN:           8.8674048604304
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         249.62729204313
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

In [121]:
weather_bbox = weather_dataset.rio.bounds()
population_dataarray = population_dataarray.sel(x=slice(weather_bbox[0], weather_bbox[2]), y=slice(weather_bbox[3], weather_bbox[1]))
population_dataarray

<xarray.DataArray (y: 3780, x: 8788)> Size: 266MB
dask.array<getitem, shape=(3780, 8788), dtype=float64, chunksize=(3780, 4096), chunktype=numpy.ndarray>
Coordinates:
  * x              (x) float64 70kB -134.1 -134.1 -134.1 ... -60.91 -60.9 -60.9
  * y              (y) float64 30kB 52.61 52.6 52.6 52.59 ... 21.14 21.13 21.12
    spatial_ref    int64 8B 0
    weather_x_idx  (x) float64 70kB -134.1 -134.1 -134.1 ... -60.91 -60.91
    weather_y_idx  (y) float64 30kB 52.6 52.6 52.6 52.57 ... 21.13 21.13 21.13
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        286330.44519772
    STATISTICS_MEAN:           8.8674048604304
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         249.62729204313
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

In [122]:
summed = population_dataarray.groupby(['weather_y_idx']).sum().groupby(['weather_x_idx']).sum()
summed

<xarray.DataArray (weather_y_idx: 1050, weather_x_idx: 2441)> Size: 21MB
dask.array<groupby_nansum, shape=(1050, 2441), dtype=float64, chunksize=(1050, 1695), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref    int64 8B 0
  * weather_y_idx  (weather_y_idx) float64 8kB 21.13 21.16 21.19 ... 52.57 52.6
  * weather_x_idx  (weather_x_idx) float64 20kB -134.1 -134.1 ... -60.94 -60.91
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        286330.44519772
    STATISTICS_MEAN:           8.8674048604304
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         249.62729204313
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

### Population coverage with subset of weather points

In [123]:
zone_bounding_box = [
    [-124.909721, 32.0341570000001],
    [-113.631212, 41.965851]
]

In [124]:
# Convert bounding box to min/max for longitude (x) and latitude (y)
min_lon, min_lat = zone_bounding_box[0]
max_lon, max_lat = zone_bounding_box[1]

# Find weather_x_idx and weather_y_idx values within the bounding box
weather_x_vals = summed.weather_x_idx.values
weather_y_vals = summed.weather_y_idx.values

# Select indices within bounding box
selected_x = weather_x_vals[(weather_x_vals >= min_lon) & (weather_x_vals <= max_lon)]
selected_y = weather_y_vals[(weather_y_vals >= min_lat) & (weather_y_vals <= max_lat)]

# Select the slice from summed
summed_subset = summed.sel(weather_x_idx=selected_x, weather_y_idx=selected_y)
summed_subset

<xarray.DataArray (weather_y_idx: 331, weather_x_idx: 376)> Size: 996kB
dask.array<getitem, shape=(331, 376), dtype=float64, chunksize=(331, 376), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref    int64 8B 0
  * weather_y_idx  (weather_y_idx) float64 3kB 32.05 32.08 32.11 ... 41.92 41.95
  * weather_x_idx  (weather_x_idx) float64 3kB -124.9 -124.9 ... -113.7 -113.6
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        286330.44519772
    STATISTICS_MEAN:           8.8674048604304
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         249.62729204313
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

For a given bounding box, we want to select the weather grid points that make up X% of the population.
We'll use california (us-cal-ciso) as an example.

#### Compute cummulative sums over grids, check what share of grid points is needed to cover what share of population

In [125]:
import numpy as np

In [126]:
total_pop = summed_subset.values.sum()
flat_sorted = summed_subset.stack(grid_cell=('weather_y_idx', 'weather_x_idx')).sortby(summed_subset.stack(grid_cell=('weather_y_idx', 'weather_x_idx')), ascending=False)
cumsum_1d = flat_sorted.cumsum(dim='grid_cell')


for threshold in np.arange(0.95, 1, 0.01):
    cutoff_idx = (cumsum_1d <= total_pop*threshold).sum().values
    selected_coords = flat_sorted.grid_cell[:cutoff_idx]

    print(f"{cutoff_idx/len(flat_sorted.grid_cell)*100}% of the grid points are needed to cover {threshold*100}% of the population")
    print(f"{cutoff_idx} grid points are needed to cover {threshold*100}% of the population")

y_coords = [coord[0] for coord in selected_coords.values]
x_coords = [coord[1] for coord in selected_coords.values]

2.971331233528315% of the grid points are needed to cover 95.0% of the population
3698 grid points are needed to cover 95.0% of the population
3.3136208780613225% of the grid points are needed to cover 96.0% of the population
4124 grid points are needed to cover 96.0% of the population
3.8133958989522405% of the grid points are needed to cover 97.0% of the population
4746 grid points are needed to cover 97.0% of the population
4.621713698013756% of the grid points are needed to cover 98.0% of the population
5752 grid points are needed to cover 98.0% of the population
6.186122002956869% of the grid points are needed to cover 99.0% of the population
7699 grid points are needed to cover 99.0% of the population
100.0% of the grid points are needed to cover 100.0% of the population
124456 grid points are needed to cover 100.0% of the population


In [127]:
threshold = 0.999
cutoff_idx = (cumsum_1d <= total_pop*threshold).sum().values
selected_coords = flat_sorted.grid_cell[:cutoff_idx]

print(f"{cutoff_idx/len(flat_sorted.grid_cell)*100}% of the grid points are needed to cover {threshold*100}% of the population")
print(f"{cutoff_idx} grid points are needed to cover {threshold*100}% of the population")

y_coords = [coord[0] for coord in selected_coords.values]
x_coords = [coord[1] for coord in selected_coords.values]

11.997814488654624% of the grid points are needed to cover 99.9% of the population
14932 grid points are needed to cover 99.9% of the population


### Plotting

In [128]:
summed_subset_values = summed_subset.sel(
    weather_y_idx=xr.DataArray(y_coords, dims='points'),
    weather_x_idx=xr.DataArray(x_coords, dims='points')
)

In [129]:
import plotly.express as px

In [130]:
values = summed_subset_values.values

fig = px.scatter_mapbox(
    lat=y_coords,  # Assuming y_coords are latitudes
    lon=x_coords,  # Assuming x_coords are longitudes
    color=values,
    color_continuous_scale='viridis',
    title=f'Top {threshold*100}% Population Grid Cells',
    mapbox_style='carto-positron',  # Light map style
    zoom=2,
    width=1000,
    height=700
)

fig.update_traces(
    marker=dict(size=6, opacity=0.7),
    hovertemplate=(
        "<b>Latitude</b>: %{lat:.4f}<br>"
        "<b>Longitude</b>: %{lon:.4f}<br>"
        "<b>Population</b>: %{marker.color:,.0f}"
        "<extra></extra>"
    )
)

fig.show()

/var/folders/s3/l6dmckzj425881s03l_7t2w80000gn/T/ipykernel_60625/1985404618.py:3: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

